## 1. Initial Question



What are the factors impacting the salary of a data scientist?
What is the salary trend for data scientist? and its comparison to living cost?
What is the predicted salary for Data Scientist in next 5 years?

1. Many people thinks that tech companies pays well above average and cost of living. Is this true?
2. Many people thinks that you don’t need a degree to work in these tech companies. Is this true?
3. Are women underpaid in these companies?
4. Are non-whites paid lower than whites in these companies?
5. Does years in company affect value of granted stock and bonus?
6. For the same position or job level, does location affect base salary regardless of gender and race?


## 2. Overview and Motivation

The Harvard Business Review indicates data scientist as the sexist job of the 21st century. 

According to the Glassdoor 2022, data scientist rank the top 3 best job in America for 7 years running. This ranking is based on the career opportunites rating, number of opening jobs, and the average salaries. The median base salary is $120,000. The job opening were also spiking up from 1,736 in 2016 to 10,071 in 2022. 

Motivation?

## 3. Related Work

<b> 1. Salary Prediction in the IT Job Market with Few High-Dimensional Samples: A Spanish Case Study </b><br>
This paper study predict the salary of IT jobs using data from Tecnoempleo, an e-Recruitment website. The result shows that experience, job stability or certain roles contribute significantly to the salary. Besides, decision trees yield a bettery accuracy (84%) than voting committe based tools.

<b> 2. Gender-based salary differences in academic medicine: a retrospective review of data from six public medical centers in the Western USA </b> <br>
The paper study the effects of gender, rank and speciality of the acedemic medical centers in the Western USA. among the 799 faculty members, it is found that male are paid significantly higher than feamle in general surgery and obstetrics and gynaecology. It is estimated female are paid ~$75,000 lower than their male colleague.

## 4. Data Cleaning / Data Preprocessing

In [1]:
import pandas as pd
data = pd.read_excel('Levels_Fyi_Salary_Data.xlsx') # Read the first sheet by default

In [2]:
df = pd.DataFrame(data)

In [3]:
df.shape

(62642, 29)

### 4.1 Duplicate value handling
No duplicate data

In [4]:
df.duplicated().sum()  # no duplicate values

0

### 4.2Missing value handling

The missing proportion of each variable in the data can be analyzed by the following function<br>
Three schemes: remove missing values, fill missing values with mean, fill missing values with statistical values (max/min)<br>
In the following processing, the commented out plan is to delete the corresponding sample, and there are filled plans, which can be selected as needed.

In [5]:
def missing_percent(df):
    nan_percent = 100*(df.isnull().sum()/len(df))
    nan_percent = nan_percent[nan_percent > 0].sort_values()
    return(nan_percent)
print(missing_percent(df))

dmaid            0.003193
company          0.007982
level            0.189968
tag              1.363303
gender          31.193129
otherdetails    35.931164
Education       51.518151
Race            64.198142
dtype: float64


#### company field

In [6]:
df.company.unique() # The company field is missing and cannot be filled with common methods. It is recommended to delete it, or fill it with a new class

array(['Oracle', 'eBay', 'Amazon', ..., 'VmWare', 'SalesForce',
       'tableau software'], dtype=object)

In [21]:
# df = df[df['company'].notnull()]
# print(df.shape)  # You can see that there are 5 less data after deletion

In [7]:
df['company'] = df['company'].fillna('unknown')

#### dmaid field

Fill with mean

In [8]:
df.dmaid.isnull().sum()

2

In [9]:
df['dmaid'] = df['dmaid'].fillna(df['dmaid'].mean()) 

#### level字段
The value of the level field is non-numeric, and the number of missing data is 119. It is the same as the above company data. You can consider deleting or filling it.

In [10]:
df.level.isnull().sum()

119

In [11]:
# 删除
# df = df[df['level'].notnull()]
# print(df.shape)

In [11]:
# 填充"unknown"
df['level'] = df['level'].fillna('unknown')

#### tag field
The value of the field is non-numeric. Similarly, the number is 854, which can be deleted or filled with "unknown" or other characters.

In [12]:
df.tag

0                                   NaN
1                                   NaN
2                                   NaN
3                                   NaN
4                                   NaN
                      ...              
62637    Distributed Systems (Back-End)
62638                        Full Stack
62639                        Full Stack
62640                               iOS
62641                           ML / AI
Name: tag, Length: 62642, dtype: object

In [13]:
df.tag.isnull().sum()

854

In [15]:
# df = df[df['tag'].notnull()]
# print(df.shape)

(61788, 29)


In [14]:
df['tag'] = df['tag'].fillna('unknown')

###gender field

In [15]:
df.gender.unique()

array([nan, 'Male', 'Female', 'Other', 'Title: Senior Software Engineer'],
      dtype=object)

In [16]:
df['gender'] = df['gender'].fillna('Other')

In [18]:
df.shape

(62642, 29)

In [19]:
df[df['gender'] == 'Title: Senior Software Engineer']['gender'] = 'Other'

D:\anoconda\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#### otherdetails field
Descriptive field, can be filled with "unknown" or not

In [20]:
df.otherdetails.unique()

array([nan, 'Tons of overtime', 'PhD', ...,
       '40k cash split over 2 years. 75k rsus over 4 years. Bonus is median 10% of base per yr.',
       '65k of stocks at 5/15/40/40 vesting schedule per year. First job out of college, only internship experience prior',
       'industry hire'], dtype=object)

#### Education field
If the missing ratio is high, it can be filled in as "unknown". Subsequent analysis can extract non-unknown samples to analyze the relationship between education and salary or other variables, or make a separate category.

In [21]:
df.Education.unique()

array([nan, 'PhD', "Master's Degree", "Bachelor's Degree", 'Some College',
       'Highschool'], dtype=object)

In [22]:
df['Education'] = df['Education'].fillna('unknown')

#### Race field
Similar to Education field handling

In [23]:
df.Race.unique()

array([nan, 'White', 'Asian', 'Black', 'Two Or More', 'Hispanic'],
      dtype=object)

In [24]:
df['Race'] = df['Race'].fillna('unknown')

In [25]:
print(missing_percent(df))

otherdetails    35.931164
dtype: float64


### 4.3 Formatting
Only the timestamp field has a format inconsistency problem

In [26]:
df.head()

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education
0,2017-07-06 11:33:27,Oracle,L3,Product Manager,127000,"Redwood City, CA",1.5,1.5,unknown,107000,...,0,0,0,0,0,0,0,0,unknown,unknown
1,2017-10-06 17:11:29,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",5.0,3.0,unknown,0,...,0,0,0,0,0,0,0,0,unknown,unknown
2,2017-11-06 14:53:57,Amazon,L7,Product Manager,310000,"Seattle, WA",8.0,0.0,unknown,155000,...,0,0,0,0,0,0,0,0,unknown,unknown
3,6/17/2017 0:23:14,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",7.0,5.0,unknown,157000,...,0,0,0,0,0,0,0,0,unknown,unknown
4,6/20/2017 10:58:51,Microsoft,60,Software Engineer,157000,"Mountain View, CA",5.0,3.0,unknown,0,...,0,0,0,0,0,0,0,0,unknown,unknown


#### timestamp Time data format is not uniform

In [27]:
df.timestamp

0        2017-07-06 11:33:27
1        2017-10-06 17:11:29
2        2017-11-06 14:53:57
3          6/17/2017 0:23:14
4         6/20/2017 10:58:51
                ...         
62637    2018-09-09 11:52:32
62638      9/13/2018 8:23:32
62639     9/13/2018 14:35:59
62640     9/16/2018 16:10:35
62641      1/29/2019 5:12:59
Name: timestamp, Length: 62642, dtype: object

In [28]:
import datetime
def stand_time(time_str):
    format_use = "%Y-%m-%d %H:%M:%S"
    formats = [
        "%Y-%m-%d %H:%M:%S", "%m/%d/%Y %H:%M:%S"]
    for s in formats:
        try:
            temp_time = datetime.datetime.strptime(time_str, s)
            return datetime.datetime.strftime(temp_time,format_use)
        except:
            pass
    return None

In [29]:
df.timestamp = [stand_time(str(item)) for item in list(df.timestamp)]

In [30]:
df.timestamp

0        2017-07-06 11:33:27
1        2017-10-06 17:11:29
2        2017-11-06 14:53:57
3        2017-06-17 00:23:14
4        2017-06-20 10:58:51
                ...         
62637    2018-09-09 11:52:32
62638    2018-09-13 08:23:32
62639    2018-09-13 14:35:59
62640    2018-09-16 16:10:35
62641    2019-01-29 05:12:59
Name: timestamp, Length: 62642, dtype: object

### 4.4Get the target data subset

In [32]:
def get_year(time_str):
    temp_time = datetime.datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S")
    return temp_time.year

In [33]:
df['year'] = [get_year(item) for item in df.timestamp]

In [34]:
df_use = df[['year','company','level','totalyearlycompensation','basesalary','stockgrantvalue','bonus']]

In [36]:
df_use.sort_values(by='totalyearlycompensation')

,year,company,level,totalyearlycompensation,basesalary,stockgrantvalue,bonus
11003,2019,Microsoft,SDE I,10000,1000,1000.0,8000.0
19972,2020,TCS,Systems Engineer,10000,9000,0.0,1000.0
15110,2020,Andela,ENG 1,10000,9000,0.0,0.0
35919,2020,Tata Consultancy Services,SE,10000,7000,1000.0,1000.0
42702,2021,Red Hat,L3,10000,10000,0.0,0.0
...,...,...,...,...,...,...,...
61969,2019,Snap,L8,2500000,400000,2100.0,0.0
61990,2021,Facebook,E9,4490000,350000,4000.0,140000.0
61982,2021,Google,L10,4500000,450000,3700.0,350000.0
61967,2019,Microsoft,80,4950000,850000,3750.0,350000.0


Two ideas: 1 is to filter out the basesalary, stockgrantvalue, bonus and the same data as totalyearlycompensation；<br>
2 is based on the sum of basesalary, stockgrantvalue, and bonus to obtain totalyearlycompensation, and subsequently this variable is named total to avoid confusion;<br>
Not sure which idea you need to use, we believe that the first is to assume that the two data are accurate; the second is to think that the basesalary, stockgrantvalue, bonus data are accurate。<br>
After inspection, no inconsistency of the units mentioned in the current data was found. Among the data with low basic salary, most of the salary is less than 10,000 in the case of low level

In [37]:
df_use[df_use['totalyearlycompensation'] == (df_use['basesalary']+df_use['stockgrantvalue']+df_use['bonus'])].sort_values(by='totalyearlycompensation')

,year,company,level,totalyearlycompensation,basesalary,stockgrantvalue,bonus
55003,2021,Virtusa,Entry,10000,10000,0.0,0.0
30149,2020,ZS associates,Associate,10000,9000,0.0,1000.0
19972,2020,TCS,Systems Engineer,10000,9000,0.0,1000.0
33972,2020,Accenture,L5,10000,10000,0.0,0.0
5467,2019,Bukalapak,Designer I,10000,10000,0.0,0.0
...,...,...,...,...,...,...,...
34812,2020,Google,L9,1355000,380000,800000.0,175000.0
10697,2019,Apple,Technical Director,1425000,320000,975000.0,130000.0
15560,2020,Google,L9,1600000,400000,1000000.0,200000.0
20561,2020,Google,L10,1650000,450000,800000.0,400000.0


In [38]:
df_use['total'] = df_use['basesalary']+df_use['stockgrantvalue']+df_use['bonus']

D:\anoconda\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [39]:
df_use.sort_values(by='total')

,year,company,level,totalyearlycompensation,basesalary,stockgrantvalue,bonus,total
7806,2019,Google,Senior SWE,200000,0,0.0,0.0,0.0
5983,2019,Google,L5,300000,0,0.0,0.0,0.0
9852,2019,Lyft,Senior,245000,0,0.0,0.0,0.0
9853,2019,Lyft,Senior,245000,0,0.0,0.0,0.0
2504,2018,Intel,Grade 9,185000,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
62351,2018,Microsoft,64,250000,1659870,49850.0,31500.0,1741220.0
6259,2019,Microsoft,68,1800000,250000,1000000.0,550000.0,1800000.0
62005,2018,Airbnb,L7,675000,260000,1660000.0,0.0,1920000.0
62423,2018,Facebook,E7,850000,250000,2000000.0,61250.0,2311250.0


In [43]:
df_use.sort_values(by='basesalary')

,year,company,level,totalyearlycompensation,basesalary,stockgrantvalue,bonus,total
7806,2019,Google,Senior SWE,200000,0,0.0,0.0,0.0
5983,2019,Google,L5,300000,0,0.0,0.0,0.0
9852,2019,Lyft,Senior,245000,0,0.0,0.0,0.0
9853,2019,Lyft,Senior,245000,0,0.0,0.0,0.0
2504,2018,Intel,Grade 9,185000,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
62351,2018,Microsoft,64,250000,1659870,49850.0,31500.0,1741220.0
6259,2019,Microsoft,68,1800000,250000,1000000.0,550000.0,1800000.0
62005,2018,Airbnb,L7,675000,260000,1660000.0,0.0,1920000.0
62423,2018,Facebook,E7,850000,250000,2000000.0,61250.0,2311250.0


In [51]:
df_use[df_use['basesalary']<5000].sort_values(by='basesalary')

,year,company,level,totalyearlycompensation,basesalary,stockgrantvalue,bonus,total
1,2017,eBay,SE 2,100000,0,0.0,0.0,0.0
6978,2019,Amazon,SDE II,160000,0,0.0,0.0,0.0
6986,2019,Cvent,Lead software engineer,167000,0,0.0,0.0,0.0
6993,2019,NetApp,Level III,129000,0,0.0,0.0,0.0
6997,2019,EPAM,M2,65000,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
3716,2019,Groupon,PM 2,150000,0,0.0,0.0,0.0
3720,2019,Salesforce,Lead MTS,248000,0,0.0,0.0,0.0
3694,2019,Facebook,L7,500000,0,0.0,0.0,0.0
11003,2019,Microsoft,SDE I,10000,1000,1000.0,8000.0,10000.0


In [54]:
df_use.describe()

,year,totalyearlycompensation,basesalary,stockgrantvalue,bonus,total
count,62642.000000,6.264200e+04,6.264200e+04,6.264200e+04,62642.000000,6.264200e+04
mean,2020.062147,2.163004e+05,1.366873e+05,5.148608e+04,19334.746588,2.075081e+05
std,0.907770,1.380337e+05,6.136928e+04,8.187457e+04,26781.292040,1.351927e+05
min,2017.000000,1.000000e+04,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
25%,2020.000000,1.350000e+05,1.080000e+05,0.000000e+00,1000.000000,1.270000e+05
50%,2020.000000,1.880000e+05,1.400000e+05,2.500000e+04,14000.000000,1.830000e+05
75%,2021.000000,2.640000e+05,1.700000e+05,6.500000e+04,26000.000000,2.600000e+05
max,2021.000000,4.980000e+06,1.659870e+06,2.800000e+06,1000000.000000,3.046000e+06


In [7]:
df.to_excel('Levels_Salary_Data_2017-2022.xlsx',index=True, header=True)

<note>
    
Junior = 1 - 4 years
Mid-level = 5-9 years
Senior = 10+ years

or 
0-1 years
104 years
5-9 years
10-19 years
20 years and above

Living cost


Introduction of team?

## Appendix

<draft>
Best Jobs in America 2022 Employees' Choice (2022). Glassdoor. Retrived on May 3, 2022 from
https://www.glassdoor.com/List/Best-Jobs-in-America-LST_KQ0,20.htm

Data Scientist Salary-The Ultimate Guide for 2021 (2022). ProjectPro. Retrived on May 3, 2022 from https://www.projectpro.io/article/data-scientist-salary-the-ultimate-guide-for-2021/218

Davenport, T H. & Patil, DJ. (2012). Data Scientist: The Sexiest Job of the 21st Century. *HARVARD BUSINESS REVIEW*. Retrieved on 3 May, 2022 from https://hbr.org/2012/10/data-scientist-the-sexiest-job-of-the-21st-century

Martin, I., Mariello, A., Battiti, R. & Hernandez, J A. (2018). Salary Prediction in the IT Job Market with Few High-Dimensional Samples: A Spanish Case Study. *International Journal of Computational Intelligence Systems, 11, 1192-1209*. https://libkey.io/libraries/2611/articles/225996272/full-text-file   
    
    
Miller, H., Seckel, Ea., White, CL., et al. (2021). Gender-based salary differences in academic medicine: a retrospective review of data from six public medical centers in the Western USA. http://dx.doi.org/10.1136/bmjopen-2021-059216
    
Seaman, A. (2021). LinkedIn Jobs on the Rise: 15 opportunities that are in demand and hiring now. Retrieved on May 3, 2022 from https://www.linkedin.com/pulse/linkedin-jobs-rise-15-opportunities-demand-hiring-now-andrew-seaman/
    